In [11]:
import numpy as np
import pandas as pd
from imdb import Cinemagoer
from tqdm import tqdm
import threading
import pyspark
tqdm.pandas()
from pyspark.sql import SQLContext, SparkSession
import sys
import warnings
warnings.simplefilter(action="ignore")

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/project/spark-3.2.1-bin-hadoop3.2"

# get a spark context
import pyspark.pandas as ps

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("PySparkPostgresApp") \
    .config("spark.jars", "/project/postgresql-42.3.2.jar") \
    .getOrCreate()

spark



postgres_uri = "jdbc:postgresql://database-1.cdz0wq8d9fyh.eu-west-2.rds.amazonaws.com:5432/imdb_database"
user = "postgres"
password = "qwerty123"

def dbToSpark(dbtable):
    
    try:
        df = spark.read \
            .format("jdbc") \
            .option("url", postgres_uri) \
            .option("dbtable", dbtable) \
            .option("user", user) \
            .option("password", password) \
            .option("driver", "org.postgresql.Driver") \
            .load()
        return df

    except Exception as ex:
        return ex


def sparkToDB(df_spark,dbtable):
    try:
        df_spark.write \
            .mode("append") \
            .format("jdbc") \
            .option("url", postgres_uri) \
            .option("dbtable", dbtable) \
            .option("user", user) \
            .option("password", password) \
            .option("driver", "org.postgresql.Driver") \
            .save()
    except Exception as ex:
        return ex
    
def fetchForeignKey(df,dbtable,mappingColumnOutset,mappingColumnTarget,mappedValue,newcol):
    
    df_curr = dbToSpark(dbtable)
    df_curr_pd = df_curr.toPandas()
    df[newcol] = df[mappingColumnOutset].map(df_curr_pd.set_index([mappingColumnTarget])[mappedValue])
    df.drop(columns=[mappingColumnOutset], inplace=True)
    return df

In [4]:
def writeToDB(movies,moviePersons,movieGenres):
    moviesDF = pd.DataFrame(movies)
    moviePersonsDF = pd.concat(moviePersons)
    movieGenresDF = pd.concat(movieGenres)
    
    moviesDF_spark = spark.createDataFrame(moviesDF)
    sparkToDB(moviesDF_spark,"imdb_schema.movie_primary")
    
    genresPresent_df_spark = dbToSpark("imdb_schema.genres")
    genresPresent_df = genresPresent_df_spark.toPandas()
    newGenres = list(set(movieGenresDF['genre_name'])-set(genresPresent_df['genre_name']))
    if len(newGenres)>0:
        newGenresDF = pd.DataFrame()
        newGenresDF['genre_name'] = newGenres
        newGenresDF_spark = spark.createDataFrame(newGenresDF)
        sparkToDB(newGenresDF_spark,"imdb_schema.genres")
    
    personPresent_df_spark = dbToSpark("imdb_schema.person_data")
    personPresent_df = personPresent_df_spark.toPandas()
    newPersons = list(set(moviePersonsDF['person_name'])-set(personPresent_df['person_name']))
    if len(newPersons)>0:
        newPersonsDF = pd.DataFrame()
        newPersonsDF['person_name'] = newPersons
        newPersonsDF_spark = spark.createDataFrame(newPersonsDF)
        sparkToDB(newPersonsDF_spark,"imdb_schema.person_data")
    
    rolesPresent_df_spark = dbToSpark("imdb_schema.role_data")
    rolesPresent_df = rolesPresent_df_spark.toPandas()
    newRoles = list(set(moviePersonsDF['role_name'])-set(rolesPresent_df['role_name']))
    if len(newRoles)>0:
        newRolesDF = pd.DataFrame()
        newRolesDF['role_name'] = newRoles
        newRolesDF_spark = spark.createDataFrame(newRolesDF)
        sparkToDB(newRolesDF_spark,"imdb_schema.role_data")
    
    movieGenresDF = fetchForeignKey(movieGenresDF,"imdb_schema.genres","genre_name",
                                    "genre_name","genre_id","genre_id")
    movieGenresDF_spark = spark.createDataFrame(movieGenresDF)
    sparkToDB(movieGenresDF_spark,"imdb_schema.movie_genres")
    
    moviePersonsDF =fetchForeignKey(moviePersonsDF,"imdb_schema.person_data","person_name",
                                    "person_name","person_id","person_id")
    moviePersonsDF =fetchForeignKey(moviePersonsDF,"imdb_schema.role_data","role_name",
                                "role_name","role_id","role_id")
    moviePersonsDF_spark = spark.createDataFrame(moviePersonsDF)
    sparkToDB(moviePersonsDF_spark,"imdb_schema.movie_person")



In [14]:
#global movies, moviePersons, movieGenres
movies = []#[None]*100
moviePersons = []
movieGenres = []
rolesList = ['cast','director','writer']

In [15]:
ia = Cinemagoer()
def fetchData(x):
    try:
        movieDict = {}
        moviePersonDF = pd.DataFrame(columns = ['movie_id','person_name','role_name'])
        genresDF = pd.DataFrame()
        movieID = "0000000" + str(x)
        movieID = movieID[-7:]
        movie = ia.get_movie(movieID)
        movieDict['movie_id'] = movieID
        movieDetails = {'plot_outline':'plot outline','movie_name':'title',
                        'movie_runtime':'runtimes','rating':'rating'}
        for key,value in movieDetails.items():
            try:
                movieDict[key] = movie.get(value)
            except:
                movieDict[key] = ''
            if type(movieDict[key]) == list:
                movieDict[key] = movieDict[key][0]
        if (movieDict['plot_outline']=='') or (movieDict['plot_outline']==None):
            return ''
        #print(movieDict)
        #movies[x-1]=movieDict
        infoAvailable = set(movie.infoset2keys['main'])
        rolesAvaliable = list(set(rolesList).intersection(infoAvailable))
        for role in rolesAvaliable:
            roleDF = pd.DataFrame()
            persons = movie.get(role)
            persons = [str(person) for person in persons]
            try:
                if role == 'cast':
                    persons = persons[:5]
            except:
                pass
            roleDF['movie_id'] = [movieID]*len(persons)
            roleDF['role_name'] = [role]*len(persons)
            roleDF['person_name'] = persons
            moviePersonDF = moviePersonDF.append(roleDF)
        genres = movie.get('genres')
        genresDF['movie_id'] = [movieID]*len(genres)
        genresDF['genre_name'] = genres
        movies.append(movieDict)
        moviePersons.append(moviePersonDF)
        movieGenres.append(genresDF)
        return movieDict,moviePersonDF,genresDF
    except:
        return ''
#     if x%1000==0:
#         writeToDB(movies,moviePersons,movieGenres)
#         movies = []
#         moviePersons = []
#         movieGenres = []

In [44]:
startList = list(range(80001,500000,100))
startList[:5]
endList =  list(range(80100,500100,100))
endList[:5]

[80100, 80200, 80300, 80400, 80500]

In [45]:
pool_size = 16
from multiprocessing.pool import ThreadPool
from timeit import default_timer as timer
import time
pool = ThreadPool(processes=pool_size)
start = timer()
for start,end in tqdm(zip(startList,endList)):
    try:
        for x in range(start,end+1):
            #fetchData(x)
            results = pool.map_async(fetchData, (x,))
        results.get()
        writeToDB(movies,moviePersons,movieGenres)
        movies = []
        moviePersons = []
        movieGenres = []
        time.sleep(3)
    except HTTPError:
        time.sleep(60)
        
pool.close()
pool.join()
end = timer()
    
print("Total time for code to execute:", end - start)
#futures = [executor.submit(try_my_operation, item) for item in items]
#concurrent.futures.wait(futures)

0it [00:00, ?it/s]2022-04-15 16:36:37,941 CRITICAL [imdbpy] /opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0080001/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 503: 'Service Unavailable'>},); kwds: {}
Traceback (most recent call last):
  File "/opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/parser/http/__init__.py", line 221, in retrieve_unicode
    response = uopener.open(url)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 640, in http_response
    response = self.parent.error(
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "/opt/anaconda/envs/Python3/lib/p

2022-04-15 16:36:37,949 CRITICAL [imdbpy] /opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0080009/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 503: 'Service Unavailable'>},); kwds: {}
Traceback (most recent call last):
  File "/opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/parser/http/__init__.py", line 221, in retrieve_unicode
    response = uopener.open(url)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 640, in http_response
    response = self.parent.error(
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/re

CRITICAL:imdbpy:IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0080002/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 503: 'Service Unavailable'>},); kwds: {}
Traceback (most recent call last):
  File "/opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/parser/http/__init__.py", line 221, in retrieve_unicode
    response = uopener.open(url)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 640, in http_response
    response = self.parent.error(
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 502, in _call_chain
    result = func(*args)
  File "/opt/anaconda/envs/Python3/lib/pytho

2022-04-15 16:36:37,952 CRITICAL [imdbpy] /opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0080011/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 503: 'Service Unavailable'>},); kwds: {}
Traceback (most recent call last):
  File "/opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/parser/http/__init__.py", line 221, in retrieve_unicode
    response = uopener.open(url)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 640, in http_response
    response = self.parent.error(
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/re

CRITICAL:imdbpy:IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0080010/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 503: 'Service Unavailable'>},); kwds: {}
Traceback (most recent call last):
  File "/opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/parser/http/__init__.py", line 221, in retrieve_unicode
    response = uopener.open(url)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 640, in http_response
    response = self.parent.error(
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 502, in _call_chain
    result = func(*args)
  File "/opt/anaconda/envs/Python3/lib/pytho

...

CRITICAL:imdbpy:caught an exception retrieving or parsing "main" info set for mopID "0080098" (accessSystem: http)
Traceback (most recent call last):
  File "/opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/parser/http/__init__.py", line 221, in retrieve_unicode
    response = uopener.open(url)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 640, in http_response
    response = self.parent.error(
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 502, in _call_chain
    result = func(*args)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 649, in http_error_default
    raise HTTPError(req.full_url, code, msg, hdrs, fp)
urllib.error.HTTPError: HTTP Error 503: Ser

0it [00:00, ?it/s]
2022-04-15 16:36:38,671 CRITICAL [imdbpy] /opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0080073/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 503: 'Service Unavailable'>},); kwds: {}
Traceback (most recent call last):
  File "/opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/parser/http/__init__.py", line 221, in retrieve_unicode
    response = uopener.open(url)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 640, in http_response
    response = self.parent.error(
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "/opt/anaconda/envs/Python3/lib/

NameError: name 'HTTPError' is not defined

2022-04-15 16:36:38,680 CRITICAL [imdbpy] /opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/__init__.py:833: caught an exception retrieving or parsing "main" info set for mopID "0080073" (accessSystem: http)
Traceback (most recent call last):
  File "/opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/parser/http/__init__.py", line 221, in retrieve_unicode
    response = uopener.open(url)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 640, in http_response
    response = self.parent.error(
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 502, in _call_chain
    result = func(*args)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 649, in http_error_def

CRITICAL:imdbpy:caught an exception retrieving or parsing "main" info set for mopID "0080088" (accessSystem: http)
Traceback (most recent call last):
  File "/opt/anaconda/envs/Python3/lib/python3.8/site-packages/imdb/parser/http/__init__.py", line 221, in retrieve_unicode
    response = uopener.open(url)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 640, in http_response
    response = self.parent.error(
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 502, in _call_chain
    result = func(*args)
  File "/opt/anaconda/envs/Python3/lib/python3.8/urllib/request.py", line 649, in http_error_default
    raise HTTPError(req.full_url, code, msg, hdrs, fp)
urllib.error.HTTPError: HTTP Error 503: Ser

In [23]:
#writeToDB(movies,moviePersons,movieGenres)
#total_successes.get()

In [24]:
ia = Cinemagoer()
movie = ia.get_movie('0094226')#, info=['taglines', 'plot'])
movie.infoset2keys

{'main': ['localized title',
  'cast',
  'genres',
  'runtimes',
  'countries',
  'country codes',
  'language codes',
  'color info',
  'aspect ratio',
  'sound mix',
  'box office',
  'certificates',
  'original air date',
  'rating',
  'votes',
  'cover url',
  'imdbID',
  'plot outline',
  'languages',
  'title',
  'year',
  'kind',
  'original title',
  'director',
  'writer',
  'producer',
  'composer',
  'cinematographer',
  'editor',
  'editorial department',
  'casting director',
  'art direction',
  'set decoration',
  'costume designer',
  'make up',
  'production manager',
  'assistant director',
  'art department',
  'sound crew',
  'special effects',
  'visual effects',
  'stunt performer',
  'camera and electrical department',
  'casting department',
  'costume department',
  'location management',
  'music department',
  'script department',
  'transportation department',
  'miscellaneous crew',
  'akas',
  'production companies',
  'distributors',
  'special effects co

In [18]:
movieGenres

[  movie_id   genre_name
 0  0000001  Documentary
 1  0000001        Short,
   movie_id genre_name
 0  0000006      Short,
   movie_id genre_name
 0  0000005      Short
 1  0000005     Comedy,
   movie_id genre_name
 0  0000002  Animation
 1  0000002      Short,
   movie_id genre_name
 0  0000007      Short
 1  0000007      Sport,
   movie_id genre_name
 0  0000003  Animation
 1  0000003     Comedy
 2  0000003      Short
 3  0000003    Romance,
   movie_id   genre_name
 0  0000008  Documentary
 1  0000008        Short,
   movie_id genre_name
 0  0000004  Animation
 1  0000004      Short,
   movie_id genre_name
 0  0000009      Short
 1  0000009    Romance,
   movie_id   genre_name
 0  0000010  Documentary
 1  0000010        Short,
   movie_id   genre_name
 0  0000012  Documentary
 1  0000012        Short,
   movie_id   genre_name
 0  0000011  Documentary
 1  0000011        Short,
   movie_id genre_name
 0  0000015  Animation
 1  0000015      Short,
   movie_id   genre_name
 0  0000017 

In [ ]:
movieDF = pd.DataFrame(movies)

In [ ]:
movieDF.head()

In [ ]:
100%51

In [ ]:
pd.concat(moviePersons)

In [ ]:
movies
movieDF = pd.DataFrame(movies)
#movieDF['movie_id'] = movieDF['movie_id'].apply(lambda x:str(x))
movieDF_spark = spark.createDataFrame(movieDF)


In [ ]:
movieDF.dtypes

In [ ]:
movieDF_spark.write \
            .mode("append") \
            .format("jdbc") \
            .option("url", postgres_uri) \
            .option("dbtable", "imdb_schema.movie_primary") \
            .option("user", user) \
            .option("password", password) \
            .option("driver", "org.postgresql.Driver") \
            .save()

In [13]:
x = []
def foo():
    x.append(1)
    
foo()
x

[1]

In [14]:
foo()
x

[1, 1]

In [9]:
pd.DataFrame(movies)

,movie_id,movie_name,movie_runtime,rating,plot_outline
0,0000007,Corbett and Courtney Before the Kinetograph,1,5.5,James J. Corbett and Peter Courtney meet in a ...
1,0000001,Carmencita,1,5.7,"Presumably, the first woman ever to appear in ..."
2,0000008,Edison Kinetoscopic Record of a Sneeze,1,5.4,A man (Edison's assistant) takes a pinch of sn...
3,0000003,Poor Pierrot,4,6.5,None
4,0000005,Blacksmith Scene,1,6.2,A stationary camera looks at a large anvil wit...
...,...,...,...,...,...
994,0000995,One Touch of Nature,10,7.3,Policeman John Murray is the proud father of a...
995,0000997,The Open Gate,11,6.7,"Hetty is engaged to George, but after her sist..."
996,0000998,Othello,12,5.6,None
997,0000999,Paul Wang's Destiny,None,5.4,None


In [21]:
print(moviePersons)

[  movie_id    person_name role_name
0  0000003  Émile Reynaud  director,   movie_id           person_name role_name
0  0000001  William K.L. Dickson  director
0  0000001            Carmencita      cast,   movie_id           person_name role_name
0  0000006  William K.L. Dickson  director,   movie_id           person_name role_name
0  0000007  William K.L. Dickson  director
1  0000007         William Heise  director
0  0000007      James J. Corbett      cast
1  0000007        Peter Courtney      cast,   movie_id    person_name role_name
0  0000004  Émile Reynaud  director,   movie_id           person_name role_name
0  0000005  William K.L. Dickson  director
0  0000005        Charles Kayser      cast
1  0000005              John Ott      cast,   movie_id    person_name role_name
0  0000002  Émile Reynaud  director,   movie_id           person_name role_name
0  0000008  William K.L. Dickson  director
0  0000008              Fred Ott      cast,   movie_id        person_name role_name
0  0

In [12]:
# moviesDF_spark = dbToSpark("imdb_schema.movie_primary")
# moviesDF = moviesDF_spark.toPandas()

movieGenresDF_spark = dbToSpark("imdb_schema.movie_genres")
movieGenresDF = movieGenresDF_spark.toPandas()

genresDF_spark= dbToSpark("imdb_schema.genres")
genresDF = genresDF_spark.toPandas()

roles_data_df_spark = dbToSpark("imdb_schema.role_data")
roles_data_df = roles_data_df_spark.toPandas()

moviePersonsDF_spark = dbToSpark("imdb_schema.movie_person")
moviePersonsDF = moviePersonsDF_spark.toPandas()

person_data_df_spark = dbToSpark("imdb_schema.person_data")
person_data_df = person_data_df_spark.toPandas()

# moviesDF['genre_id'] = moviesDF['movie_id'].map(movieGenresDF.set_index(['movie_id'])['genre_id'])
# moviesDF['genre_name'] = moviesDF['genre_id'].map(genresDF.set_index(['genre_id'])['genre_name'])



In [15]:
sparkToDB(movieGenresDF_spark,"imdb_schema.movie_genres")

In [6]:
moviesDF_spark.write \
            .mode("append") \
            .format("jdbc") \
            .option("url", postgres_uri) \
            .option("dbtable", "imdb_schema.movie_primary") \
            .option("user", user) \
            .option("password", password) \
            .option("driver", "org.postgresql.Driver") \
            .save()

Py4JJavaError: An error occurred while calling o66.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 4) (cube-19972c0e-440b-44b4-9df4-d9f5c4fc2d36-b488f9bbf-m9tzm executor driver): net.razorvine.pickle.PickleException: problem construction object: java.lang.reflect.InvocationTargetException
	at net.razorvine.pickle.objects.AnyClassConstructor.construct(AnyClassConstructor.java:29)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:773)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:213)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:123)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:136)
	at org.apache.spark.api.python.SerDeUtil$.$anonfun$pythonToJava$2(SerDeUtil.scala:121)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterato

In [9]:
movieGenresDF = movieGenresDF_spark.toPandas()
genresDF = genresDF_spark.toPandas()
#roles_data_df = roles_data_df_spark.toPandas()
#moviePersonsDF = moviePersonsDF_spark.toPandas()

In [10]:
genresDF.shape

(28, 2)

In [59]:
person_data_df_spark = dbToSpark("imdb_schema.person_data")
person_data_df = person_data_df_spark.toPandas()

In [2]:
from postgres_wrapper import PGWrapper
config = {
    "database": "imdb_database",
    "user": "postgres",
    "password": "qwerty123",
    "host": "database-1.cdz0wq8d9fyh.eu-west-2.rds.amazonaws.com",
    "port": 5432,
}

db_con = PGWrapper(config)
db_con.connect()
results = db_con.query_as_list_of_dicts("SELECT * FROM imdb_schema.movie_primary")

In [3]:
moviesDF=pd.DataFrame(results)

In [16]:
def fetchGenres(movieID):
    movieGenres = movieGenresDF[movieGenresDF['movie_id']==movieID]
    movieGenres['genre_name']= movieGenres['genre_id'].map(genresDF.set_index(['genre_id'])['genre_name'])
    return movieGenres['genre_name'].tolist()

In [17]:
moviesDF['genres'] = moviesDF['movie_id'].progress_apply(lambda x: fetchGenres(x))

In [18]:
moviesDF1 = moviesDF[moviesDF['movie_runtime'].notna()]
len(moviesDF1)

28790

In [22]:
for genre in tqdm(genresDF['genre_name'].tolist()):
    moviesDF1[genre] = moviesDF1.apply(lambda row: 1 if genre in row['genres'] else 0, axis = 1)

100%|██████████| 28/28 [00:05<00:00,  4.91it/s]


In [19]:
# moviesDF1.to_excel('finalData.xlsx')

In [28]:
!pip install flair==0.10
    
import flair
from flair.data import Sentence
from flair.embeddings import WordEmbeddings, ELMoEmbeddings, TransformerWordEmbeddings, TransformerDocumentEmbeddings


         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.                  

• Project files and data should be stored in /project. This is shared among everyone
  in the project.
• Personal files and configuration should be stored in /home/faculty.
• Files outside /project and /home/faculty will be lost when this server is terminated.
• Create custom environments to setup your servers reproducibly.

  Using cached flair-0.10-py3-none-any.whl (322 kB)
     |████████████████████████████████| 77 kB 589 kB/s  eta 0:00:01
  Using cached Janome-0.4.2-py2.py3-none-any.whl (19.7 MB)
  Using cached Deprecated-1.2.13-py2.py3-none-any.whl (9.6 kB)
     |████████████████████████████████| 4.0 MB 41.6 MB/s eta 0:00:01
  Using cached torc

In [31]:
glove_embedding = WordEmbeddings('glove')
embedding_size = glove_embedding.embedding_length
sentence1 = Sentence(moviesDF1.iloc[0]['plot_outline'])
glove_embedding.embed(sentence1)


[Sentence: "A stationary camera looks across the boulevard at a diagonal toward one corner of Lyon 's Cordeliers' Square . It 's a long shot , with a great deal of depth of focus . We can see the sky and fronts of four buildings , each four or five stories tall . It 's a busy thoroughfare , with pedestrians walking in front of the buildings and crossing the boulevard between horse-drawn vehicles . A double-decker bus passes in front of us , pulled by two horses . Various tradesmen pass on wagons . One van passes ."   [− Tokens: 98]]

In [132]:
moviesDF1.iloc[0]['plot_outline']

"A stationary camera looks across the boulevard at a diagonal toward one corner of Lyon's Cordeliers' Square. It's a long shot, with a great deal of depth of focus. We can see the sky and fronts of four buildings, each four or five stories tall. It's a busy thoroughfare, with pedestrians walking in front of the buildings and crossing the boulevard between horse-drawn vehicles. A double-decker bus passes in front of us, pulled by two horses. Various tradesmen pass on wagons. One van passes."

In [54]:
Sentence(moviesDF1.iloc[0]['plot_outline'])

Sentence: "A stationary camera looks across the boulevard at a diagonal toward one corner of Lyon 's Cordeliers' Square . It 's a long shot , with a great deal of depth of focus . We can see the sky and fronts of four buildings , each four or five stories tall . It 's a busy thoroughfare , with pedestrians walking in front of the buildings and crossing the boulevard between horse-drawn vehicles . A double-decker bus passes in front of us , pulled by two horses . Various tradesmen pass on wagons . One van passes ."   [− Tokens: 98]

In [33]:
np.array([np.mean([sentence1[m].embedding[x] for m in range(len(sentence1))]) for x in range(len(sentence1[0].embedding))])

array([-7.94622526e-02,  6.82952255e-02,  2.72786260e-01, -1.86349675e-01,
        2.11012915e-01,  2.61416227e-01, -8.80796313e-02,  3.07564676e-01,
       -1.72657773e-01,  5.67534342e-02,  1.70402780e-01, -4.77393456e-02,
        2.19378367e-01,  7.59656131e-02,  2.92817414e-01, -1.79668128e-01,
        1.81592837e-01,  9.34086740e-03, -3.31881613e-01, -1.34457706e-03,
        4.23103988e-01,  6.31012693e-02,  2.22750857e-01, -2.87026633e-02,
        4.70186204e-01, -1.84090346e-01, -1.59299076e-01, -2.95479923e-01,
       -5.20426147e-02, -1.36627913e-01, -1.22507505e-01,  3.15979421e-01,
       -1.54714193e-03,  3.13626160e-03,  6.33994788e-02,  1.76162660e-01,
        1.28738672e-01,  2.33258501e-01,  8.14920813e-02,  9.89636183e-02,
       -2.37983450e-01, -3.39531958e-01,  2.06458211e-01, -1.73702806e-01,
        2.96250116e-02,  7.71811232e-02,  1.63596585e-01, -2.25666985e-01,
       -5.14240563e-02, -3.41994643e-01,  1.41412085e-02, -9.12926272e-02,
        6.06125034e-02,  

In [96]:
def vectorizeSentence(sentence):
    glove_embedding.embed(sentence)
    return np.array([np.mean([sentence[m].embedding[x] for m in range(len(sentence))]) for x in range(len(sentence[0].embedding))])

moviesDF1['name_plot'] = moviesDF1.apply(lambda row: row['movie_name']+' - '+row['plot_outline'],axis = 1)
moviesDF1['name_plot_vectorized'] = moviesDF1['name_plot'].progress_apply(lambda x: vectorizeSentence(Sentence(x)))
#moviesDF1['name_vectorized'] = moviesDF1['movie_name'].progress_apply(lambda x: vectorizeSentence(Sentence(x)))

100%|██████████| 28790/28790 [51:38<00:00,  9.29it/s]  


In [97]:
moviesDF1.head(100)

,movie_id,movie_name,movie_runtime,plot_outline,rating,genres,Romance,News,Animation,Comedy,...,Talk-Show,Reality-TV,Adult,plot_vectorized,name_vectorized,normalized_rating,plot_sentence,runtime_normalized,name_plot,name_plot_vectorized
0,0000027,Cordeliers' Square in Lyon,1,A stationary camera looks across the boulevard...,5.6,"[Documentary, Short]",0,0,0,0,...,0,0,0,"[-0.07946225, 0.068295226, 0.27278626, -0.1863...","[0.27884075, -0.026810005, -0.066925496, 0.129...",0.12,"[(Token: 1 A, Token: 2 stationary, Token: 3 ca...",0.000000,Cordeliers' Square in Lyon - A stationary came...,"[-0.07696736, 0.069864586, 0.26245904, -0.1816..."
2,0000021,The Photographical Congress Arrives in Lyon,1,Members of the French Photographic Society arr...,5.7,"[Documentary, Short]",0,0,0,0,...,0,0,0,"[-0.05347773, 0.1110159, 0.20198119, -0.203555...","[-0.047319163, -0.15532734, 0.54127467, -0.036...",0.14,"[(Token: 1 Members, Token: 2 of, Token: 3 the,...",0.000000,The Photographical Congress Arrives in Lyon - ...,"[-0.069014676, 0.09636637, 0.23400398, -0.2003..."
3,0000020,The Derby 1895,1,"A stationary camera, looking diagonally across...",4.8,"[Documentary, Short, Sport]",0,0,0,0,...,0,0,0,"[-0.03976103, 0.062304758, 0.38596523, -0.2819...","[0.26332533, -0.09233666, -0.077725984, -0.338...",-0.04,"[(Token: 1 A, Token: 2 stationary, Token: 3 ca...",0.000000,"The Derby 1895 - A stationary camera, looking ...","[-0.046019807, 0.064021714, 0.36128956, -0.299..."
4,0000023,The Sea,1,The sea is before us. Some rocks are visible t...,5.7,"[Documentary, Short]",0,0,0,0,...,0,0,0,"[-0.06712052, 0.24126318, 0.36458135, -0.26170...","[-0.311577, -0.103966504, 0.68901503, -0.12772...",0.14,"[(Token: 1 The, Token: 2 sea, Token: 3 is, Tok...",0.000000,The Sea - The sea is before us. Some rocks are...,"[-0.0809841, 0.23860729, 0.37162447, -0.26531,..."
5,0000031,Jumping the Blanket,1,"Outdoors, with a nondescript building in the b...",5.5,"[Documentary, Short]",0,0,0,0,...,0,0,0,"[-0.06316189, 0.089918524, 0.38536254, -0.2373...","[0.17459865, -0.019626671, 0.42243338, -0.2551...",0.10,"[(Token: 1 Outdoors, Token: 2 ,, Token: 3 with...",0.000000,"Jumping the Blanket - Outdoors, with a nondesc...","[-0.06651333, 0.09125983, 0.38741693, -0.24278..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0000335,Soldiers of the Cross,40,The plot outlined the story of the early Chris...,6.1,"[Short, Biography, Drama]",0,0,0,0,...,0,0,0,"[-0.05257072, 0.13807157, 0.26515916, -0.16061...","[-0.0701985, -0.031142503, 0.070279986, 0.0551...",0.22,"[(Token: 1 The, Token: 2 plot, Token: 3 outlin...",0.029235,Soldiers of the Cross - The plot outlined the ...,"[-0.06530712, 0.13602589, 0.26039174, -0.16003..."
104,0000340,Uncle Josh in a Spooky Hotel,1,Another adventure involving the recurring char...,4.9,"[Comedy, Fantasy, Short]",0,0,0,1,...,0,0,0,"[0.028813528, 0.049082734, 0.40425545, -0.2127...","[0.20363717, -0.1104841, 0.28002, -0.29484534,...",-0.02,"[(Token: 1 Another, Token: 2 adventure, Token:...",0.000000,Uncle Josh in a Spooky Hotel - Another adventu...,"[0.025189344, 0.042988524, 0.39530563, -0.2319..."
105,0000341,Uncle Josh's Nightmare,2,"Poor Uncle Josh is trying to get to sleep, but...",4.9,"[Comedy, Short, Fantasy]",0,0,0,1,...,0,0,0,"[-0.10433494, 0.08680299, 0.21397837, -0.26974...","[0.485565, -0.047040164, 0.65238, -0.884985, -...",-0.02,"[(Token: 1 Poor, Token: 2 Uncle, Token: 3 Josh...",0.000750,Uncle Josh's Nightmare - Poor Uncle Josh is tr...,"[-0.07412906, 0.0865076, 0.2666673, -0.3486767..."
106,0000337,Spanish Bullfight,1,"With a crowded arena in the background, a stat...",5,"[Short, Sport]",0,0,0,0,...,0,0,0,"[-0.028000712, 0.03543403, 0.26265943, -0.2142...","[0.50072, -0.1327454, -0.21441849, 0.220693, -...",0.00,"[(Token: 1 With, Token: 2 a, Token: 3 crowded,...",0.000000,Spanish Bullfight - With a crowded arena in th...,"[-0.030703211, 0.03922082, 0.25235966, -0.2124..."


In [98]:
X = moviesDF1[['name_plot_vectorized']]

In [99]:
y = moviesDF1[genresDF['genre_name'].tolist()]

In [113]:
y

,Romance,News,Animation,Comedy,Sport,Documentary,Horror,Short,Western,Biography,...,History,Music,Mystery,Thriller,Musical,Film-Noir,Game-Show,Talk-Show,Reality-TV,Adult
0,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30182,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30183,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30184,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30185,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [101]:
from sklearn.model_selection import train_test_split

X_pretrain, X_test, y_pretrain, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


In [120]:
lgb_clf = lgb.LGBMClassifier()
lgb_clf.fit(X_train['name_plot_vectorized'].to_list(), y_train)

LGBMClassifier()

In [93]:
from sklearn.metrics import accuracy_score, f1_score,confusion_matrix
y_pred = lgb_clf.predict(X_val['plot_vectorized'].to_list())
accuracy_dict = accuracy_score(y_val, y_pred)*100
accuracy_dict
confusion_matrix(y_val, y_pred)

array([[3144,   61],
       [ 640,   42]])

In [115]:
type(y_val)

pandas.core.series.Series

In [128]:
import lightgbm as lgb
import pickle
accuracy_dict,cm_dict = {},{}
for label in tqdm(genresDF['genre_name'].tolist()):
    X_train, X_val, y_train, y_val = train_test_split(X_pretrain, y_pretrain[label], test_size = 0.15, 
                                                      random_state = 42, stratify=y_pretrain[label])
    

    
    lgb_clf = lgb.LGBMClassifier(boosting_type= 'gbdt',
        objective= 'binary',
        num_leaves= 20,
        learning_rate=0.05,
        feature_fraction= 0.9,
        bagging_fraction= 0.8,
        bagging_freq= 5,
        )
    
    lgb_clf.fit(X_train['name_plot_vectorized'].to_list(), y_train)
    y_pred = lgb_clf.predict(X_test['name_plot_vectorized'].to_list())
    accuracy_dict[label] = accuracy_score(y_test[label], y_pred)*100
    cm_dict[label] = confusion_matrix(y_test[label], y_pred)
    pickle.dump(lgb_clf, open(r"/project/imdb_genrePredictor/models/PICKLE_"+str(label)+".pkl", "wb"))
    
    

  4%|▎         | 1/28 [00:00<00:18,  1.43it/s]

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8


  7%|▋         | 2/28 [00:01<00:16,  1.58it/s]

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8


 11%|█         | 3/28 [00:01<00:16,  1.53it/s]

...

 93%|█████████▎| 26/28 [00:17<00:01,  1.56it/s]

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8


 96%|█████████▋| 27/28 [00:18<00:00,  1.59it/s]

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8


100%|██████████| 28/28 [00:18<00:00,  1.49it/s]


In [131]:
accuracy_dict

{'Romance': 82.73706147968043,
 'News': 99.23584577978465,
 'Animation': 96.49183744355679,
 'Comedy': 75.79020493226814,
 'Sport': 98.54116012504342,
 'Documentary': 97.95067731851337,
 'Horror': 95.484543244182,
 'Short': 93.40048627995832,
 'Western': 93.57415769364363,
 'Biography': 97.74227162209101,
 'Crime': 89.26710663424801,
 'Fantasy': 97.67280305661689,
 'Drama': 65.43938867662384,
 'Family': 93.40048627995832,
 'Sci-Fi': 97.49913164293157,
 'Action': 91.3511635984717,
 'Adventure': 91.49010072941994,
 'War': 96.3876345953456,
 'History': 96.76971170545329,
 'Music': 97.15178881556096,
 'Mystery': 94.12990621743661,
 'Thriller': 91.49010072941994,
 'Musical': 95.51927752691907,
 'Film-Noir': 98.33275442862104,
 'Game-Show': 99.89579715178881,
 'Talk-Show': 99.96526571726294,
 'Reality-TV': 99.75686002084056,
 'Adult': 99.58318860715526}

In [13]:
postgres_uri = "jdbc:postgresql://database-1.cdz0wq8d9fyh.eu-west-2.rds.amazonaws.com:5432/imdb_database"

In [4]:
moviesDF_spark = spark.createDataFrame(moviesDF)

In [7]:
moviesDF

,movie_id,movie_name,movie_runtime,plot_outline,rating
0,0000027,Cordeliers' Square in Lyon,1,A stationary camera looks across the boulevard...,5.6
1,0000025,The Oxford and Cambridge University Boat Race,None,Although the content of this film is primitive...,4.3
2,0000021,The Photographical Congress Arrives in Lyon,1,Members of the French Photographic Society arr...,5.7
3,0000020,The Derby 1895,1,"A stationary camera, looking diagonally across...",4.8
4,0000023,The Sea,1,The sea is before us. Some rocks are visible t...,5.7
...,...,...,...,...,...
30182,0079709,Penitentiary,99,A hitchhiker named Martel Gordone gets in a fi...,5.6
30183,0079707,Portable Country,103,"On board of a bus, Andrés Barazarte travels th...",7.1
30184,0079714,Phantasm,89,"Mike, a young teenage boy who has just lost hi...",6.6
30185,0079712,Perro callejero II,96,"After serving time in prison, ""Perro"", the tit...",7.3


In [9]:
config = {
    "database": "imdb_database",
    "user": "postgres",
    "password": "qwerty123",
    "host": "database-1.cdz0wq8d9fyh.eu-west-2.rds.amazonaws.com",
    "port": 5432,
}

db_con = PGWrapper(config)
db_con.connect()

for index, row in tqdm(moviesDF.iterrows()):
    query = "INSERT INTO imdb_schema.movie_primary (movie_id, movie_name, movie_runtime,plot_outline,rating) VALUES (%s, %s, %s, %s, %s)"
    db_con.cursor.execute(query,(str(row.movie_id),str(row.movie_name), str(row.movie_runtime),str(row.plot_outline),row.rating))

30187it [18:19, 27.45it/s]
